In [ ]:
from typing import List
import itertools
import nibabel as nib
import numpy as np

def extract_roi_name(nifti_path: str) -> str:
    
    assert '_roi-' in nifti_path
    
    return str(nifti_path).split('_roi-')[1].split('.nii.gz')[0]

def get_empty_template(nifti_path: str) -> np.ndarray:

    return np.zeros(nib.load(nifti_path).shape)

def create_roi_array(roi_index, roi_path, hemi_path, hemi_idx) -> np.ndarray:

    # Load data
    roi_data = (nib.load(roi_path).get_fdata() > 0).astype(int)
    hemi_data = nib.load(hemi_path).get_fdata()
    
    # Remove opposite hemisphere depending on label
    hemi_data[np.where(hemi_data != hemi_idx)] = 0
    hemi_data[np.where(hemi_data == hemi_idx)] = 1
    
    # Filter roi data with hemisphere
    roi_data = roi_data * hemi_data

    # Label roi data
    roi_data *= roi_index
    
    return roi_data
    
    

def create_atlas(roi_nifti_list: List, hemi_nifti: str, out_dir: str = "/tmp/mouse_atlas.nii.gz"):
    
    # `hemi_nifti` annotates RH with idx == 2 and LH with idx == 1
    hemi_mapping = {
        'RH': 2,
        'LH': 1,
    }

    # Instantiate empty atlas array (to be filled in)
    merged_atlas = get_empty_template(roi_nifti_list[0])
    atlas_annotations = [] # Set up empty annotation list
    
    for ix, (roi_nifti, hemi_label) in enumerate(itertools.product(roi_nifti_list, hemi_mapping.keys())):

        ix = ix + 1
        hemi_idx = hemi_mapping[hemi_label]
        
        # All niftis must be the same shape as the empty atlas array
        assert nib.load(roi_nifti).shape == merged_atlas.shape

        # Add reannotated ROI to a single array
        merged_atlas += create_roi_array(ix, roi_nifti, hemi_nifti, hemi_idx)
        # Add label to atlas annotations
        roi_label = extract_roi_name(roi_nifti)
        atlas_annotations.append( (ix, f"{roi_label}_{hemi_label}") )

    # Round
    merged_atlas = np.round(merged_atlas)
    
    # Save
    template_img = nib.load(roi_nifti_list[0])
    merged_atlas_img = nib.Nifti1Image(
        merged_atlas, 
        affine=template_img.affine, 
        header = template_img.header
    )
    nib.save(merged_atlas_img, out_dir)

    return atlas_annotations, out_dir

def check_labels_in_atlas(atlas_annots, atlas_path):

    import nibabel as nib
    atlas_data = np.round(nib.load(atlas_path).get_fdata())
    unique_labels = np.unique(atlas_data)
    for roi_idx, roi_label in atlas_annots:
        if roi_idx not in unique_labels:
            print(f"[WARNING] {roi_label} [{roi_idx}] not in atlas.")

In [ ]:
hemi_nifti = "/opt/animalfmritools/animalfmritools/data_template/MouseABA/roi-hemispheres_space-P56_downsample2.nii.gz" # RH == 2 & LH == 1
CH = !ls /opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/*desc-CH*
BS = !ls /opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/*desc-BS*

atlas_annots, atlas_nifti = create_atlas(CH, hemi_nifti)

In [ ]:
from pathlib import Path
bolds = !ls /opt/animalfmritools/animalfmritools/data/MouseAD/bids/derivatives/bold_preproc/sub-*/ses-*/func/*desc-denoised_bold.nii.gz


for bold in bolds:
    
    bold = Path(bold)
    sub_id = bold.stem.split('sub-')[1].split('_')[0]
    ses_id = bold.stem.split('ses-')[1].split('_')[0]
    run_id = bold.stem.split('run-')[1].split('_')[0]

    resliced_atlas_path = Path(f"/tmp/sub-{sub_id}_ses-{ses_id}_atlas.nii.gz")
    
    
    # Reslice atlas to subject's templace space
    if not resliced_atlas_path.exists():
        print(f"Reference: {bold.stem}")
        !flirt -in {atlas_nifti} -ref {bold} -out {resliced_atlas_path} -interp nearestneighbour -applyxfm -usesqform

    if run_id == '01':
        print(resliced_atlas_path)
        check_labels_in_atlas(atlas_annots, resliced_atlas_path)
 
    

In [1]:
!ls /tmp/sub*nii.gz .

/tmp/sub-CD3103F2_ses-20220601_atlas.nii.gz
/tmp/sub-CD379M2_ses-20211110_atlas.nii.gz
/tmp/sub-FGD41F3_ses-20220511_atlas.nii.gz
/tmp/sub-FGD43M3_ses-20220414_atlas.nii.gz

.:
A_denoise_preprocessed_data.ipynb      CC		     test_run.ipynb
B_calculate_connectivity_matrix.ipynb  mouse_template.ipynb
